In [ ]:
#API request to get data
import requests as req
url = 'http://api.tvmaze.com/schedule?country=US'
resp = req.get(url)
response = resp.text

In [ ]:
type(response)

In [ ]:
#Convert str datatype of response to list since json_normalize function from pandas
#requires JSON array i.e. list of records
import json
data = json.loads(response)
type(data)

In [ ]:
#Module to load data as pandas dataframe using json_normalize function from pandas
import pandas as pd
from pandas.io.json import json_normalize
df = json_normalize(data)
df.head()

In [ ]:
#get shape of generated data frame
df.shape

In [ ]:
#List all column names to select relevant ones for further data processing
df.columns

In [ ]:
#Get relevant columns from dataframe on which knn algorithm can be applied(feature selection)
df_relevant = df[['name','id','show.name','show.genres','show.rating.average','show.type']]

In [ ]:
#Print newly created data frame
df_relevant.head()

In [ ]:
'''The reason to merge name column with show.name column was because
some of the shows had repeated episodes. For example:- Abby Hatcher show had two entries with 
different episode names viz Afraid of Cats and Chef Curly. I needed unique names
to identify nieghbours of an episode correctly which is why I merged the columns to
generate unique names column'''
df_relevant["full_name"] = df_relevant["show.name"].map(str) +"-"+ df_relevant["name"]

In [ ]:
df_relevant.head()

In [ ]:
#Dropping the source columns from which full_name was generated
df_relevant = df_relevant.drop(['name','show.name'], axis=1)


In [ ]:
#rename column names of dataframe for ease of understanding
renamed_columns_dictionary = {'show.genres': 'genres', 
                              'show.language': 'language',
                              'show.rating.average':'rating',
                              'show.type':'type'
                             }
df_relevant.rename(columns=renamed_columns_dictionary, inplace=True)

In [ ]:
df_relevant

In [ ]:
'''There were many entries which had genres columns empty. We have the choice to
substitute the empty values or drop them entirely'''
df_relevant[df_relevant['genres'].str.len() == 0].shape

In [ ]:
#Dropping all columns whose genres column is empty for now 
#Adding relevant genres based on show type can be a better option later
df_relevant = df_relevant.drop(df_relevant[df_relevant['genres'].str.len() == 0].index)

In [ ]:
#Considerable number of rows were removed
df_relevant.shape

In [ ]:
#Creating a new copy for ease of use and keeping a backup just in case anything messes up ahead
df = df_relevant.copy()

In [ ]:
df.head()

In [ ]:
#Check for all null values in dataframe
df.isnull().sum(axis = 0)
#Ratings column had some null values

In [ ]:
#Check datatypes of all values
df.dtypes

In [ ]:
#Fill null values in rating column with median of all values in rating column
df["rating"].fillna(df["rating"].median(), inplace = True)

In [ ]:
df.head()

I had to do label encoding for genres column. But pd.get_dummies function in 
pandas does not take list as row values due to which I had to convert them into
string of comma separated values 

In [ ]:
df['genres'] = df['genres'].apply(lambda x: ",".join(x))

In [ ]:
df.head()

In [ ]:
# Now we can get label encoded values from genres column using get_dummies fxn from pandas
df["genres"].str.get_dummies(sep=',')

We are creating a new data frame to be used for generating model using knn. We are neglecting name and id column since they are not useful for finding neighbours. 
"Genres" and "type" column is label encoded while rating column is taken as it is

In [ ]:
tv_show_features = pd.concat([df["genres"].str.get_dummies(sep=","),
                            pd.get_dummies(df[["type"]]),
                            df[["rating"]]],axis=1)

In [ ]:
tv_show_features

Since the ratings column has values from 0 to 10 while other columns have values from
0 to 1 this can bias the distance metric in KNN because features containing bigger numbers will be weighted heavily while the other features will be discounted.
So I ended up using MinMaxScaler from scikit-learn as it scales the values from 0–1.

In [ ]:
from sklearn.preprocessing import MinMaxScaler
min_max_scaler = MinMaxScaler()
tv_show_features = min_max_scaler.fit_transform(tv_show_features)

In [ ]:
import numpy as np
np.round(tv_show_features,2)

In [ ]:
# Then we fit the KNN model from scikit learn to the data and calculate 
# the nearest neighbors for each distances.
from sklearn.neighbors import NearestNeighbors
nbrs = NearestNeighbors(n_neighbors=6, algorithm='ball_tree').fit(tv_show_features)
distances, indices = nbrs.kneighbors(tv_show_features)

In [ ]:
distances

In [ ]:
#Helper fucntions to get relevant predictions
def get_index_from_name(name):
    return df[df["full_name"]==name].index.tolist()[0]

In [ ]:
all_show_names = list(df.full_name.values)

In [ ]:
def get_id_from_partial_name(partial):
    for name in all_show_names:
        if partial in name:
            print(name,all_show_names.index(name))

In [ ]:
def print_similar_tvshows(query=None,id=None):
    if id:
        for id in indices[id][1:]:
            print(df.iloc[id]["full_name"])
    if query:
        found_id = get_index_from_name(query)
        for id in indices[found_id][1:]:
            print(df.iloc[id]["full_name"])

In [ ]:
df

In [ ]:
#Change values below as per full_name column values
get_id_from_partial_name("Days of Our Lives-Ep. #13507")

In [ ]:
get_index_from_name("Days of Our Lives-Ep. #13507")

In [ ]:
print_similar_tvshows("Gotham-Trespassers")

In [ ]:
df[df["full_name"]=="Gotham-Trespassers"]

In [ ]:
showrecs=[
"Fam-Pilot",
"The Big Bang Theory-The Propagation Proposition",
"Mom-Hacky Sack and a Beautiful Experience",
"Young Sheldon-A Tummy Ache and a Whale of a Metaphor",
"The Orville-Home"
]

df[df['full_name'].isin(showrecs)]